In [67]:
#WORKS: creating probabilistic assignments for the anatomical thalamus segmentations 

import numpy as np 
import ants 

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-19'

segmentation_path = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"

template = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"

lut_file = f"{wk_dir}/compressionLookupTable.txt"

affine_transform_path = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_0GenericAffine.mat"
warp_transform_path = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_1Warp.nii.gz"
transform_list = [warp_transform_path, affine_transform_path]

lut_label_to_name = {}
with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name
thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)    

template_img = ants.image_read(template)
seg_img = ants.image_read(segmentation_path)

warped_prob_maps_list = []

for i, label in enumerate(thalamus_labels):
    print(f"Processing label {label} ({i+1}/{K})")
    mask_3d = (seg_img == label).astype('float32')
    warped_mask_3d = ants.apply_transforms(fixed=template_img, moving=mask_3d, transformlist=transform_list, interpolator='linear')
        
    warped_prob_maps_list.append(warped_mask_3d)

warped_prob_4d_img = ants.merge_channels(warped_prob_maps_list)

prob_array_kxp = warped_prob_4d_img.numpy()

#visualizing as a nii file 

output_5d_nii = f"{wk_dir}/{sub}_5d_thalamus_prob_map.nii.gz"

ants.image_write(warped_prob_4d_img, output_5d_nii)

#column_sums = prob_array_kxp.sum(axis=0) 
#thalamus_mask_1d = column_sums > 0

#thalamus_prob_map = prob_array_kxp[:, thalamus_mask_1d] 

#K=thalamus_prob_map.shape[0]
#P=thalamus_prob_map.shape[1]

#column_sums_final = thalamus_prob_map.sum(axis=0)
#column_sums_final[column_sums_final == 0] = 1

#final_prob_map = thalamus_prob_map / column_sums_final

#np.save(f"{wk_dir}/pseg_matrices_Language7T/{sub}_pseg_matrix.npy", final_prob_map)



Processing label 8103 (1/58)
Processing label 8104 (2/58)
Processing label 8105 (3/58)
Processing label 8106 (4/58)
Processing label 8108 (5/58)
Processing label 8109 (6/58)
Processing label 8110 (7/58)
Processing label 8111 (8/58)
Processing label 8112 (9/58)
Processing label 8113 (10/58)
Processing label 8115 (11/58)
Processing label 8116 (12/58)
Processing label 8117 (13/58)
Processing label 8118 (14/58)
Processing label 8119 (15/58)
Processing label 8120 (16/58)
Processing label 8121 (17/58)
Processing label 8122 (18/58)
Processing label 8123 (19/58)
Processing label 8125 (20/58)
Processing label 8126 (21/58)
Processing label 8127 (22/58)
Processing label 8128 (23/58)
Processing label 8129 (24/58)
Processing label 8130 (25/58)
Processing label 8133 (26/58)
Processing label 8134 (27/58)
Processing label 8135 (28/58)
Processing label 8136 (29/58)
Processing label 8203 (30/58)
Processing label 8204 (31/58)
Processing label 8205 (32/58)
Processing label 8206 (33/58)
Processing label 82

In [85]:
from nilearn.image import mean_img
import nibabel as nb

wk_dir = '/Users/incehusain/fs_projects/pseg_Language7T'

subject_file = [ 'sub-01_4d_thalamus_prob_map.nii.gz', 'sub-02_4d_thalamus_prob_map.nii.gz', 'sub-03_4d_thalamus_prob_map.nii.gz',
                    'sub-04_4d_thalamus_prob_map.nii.gz', 'sub-06_4d_thalamus_prob_map.nii.gz',
                    'sub-07_4d_thalamus_prob_map.nii.gz', 'sub-08_4d_thalamus_prob_map.nii.gz', 'sub-09_4d_thalamus_prob_map.nii.gz',
                    'sub-10_4d_thalamus_prob_map.nii.gz', 'sub-12_4d_thalamus_prob_map.nii.gz',
                    'sub-13_4d_thalamus_prob_map.nii.gz', 'sub-14_4d_thalamus_prob_map.nii.gz', 'sub-15_4d_thalamus_prob_map.nii.gz',
                    'sub-16_4d_thalamus_prob_map.nii.gz', 'sub-17_4d_thalamus_prob_map.nii.gz', 'sub-18_4d_thalamus_prob_map.nii.gz',
                    'sub-19_4d_thalamus_prob_map.nii.gz' ]

N=len(subject_file)

first_img = nb.load(f"{wk_dir}/{subject_file[0]}")

sum_of_data = first_img.get_fdata().astype('float64')

for i in range(1, N):
    img = nb.load(f"{wk_dir}/{subject_file[i]}")
    sum_of_data += img.get_fdata()

mean_data = sum_of_data / N

mean_img_nii = nb.Nifti1Image(mean_data.astype('float32'), first_img.affine, first_img.header)

nb.save(mean_img_nii, f"{wk_dir}/group_mean_thalamus_prob_map.nii.gz")

#group_map_img = mean_img([f"{wk_dir}/{f}" for f in subject_file])



KeyboardInterrupt: 

In [82]:
import nibabel as nb 
import numpy as np

sub = 'sub-19'

input_5d_nii = f"{wk_dir}/{sub}_5d_thalamus_prob_map.nii.gz"

output_4d_nii = f"{wk_dir}/{sub}_4d_thalamus_prob_map.nii.gz"

img_5d = nb.load(input_5d_nii)

data_5d = img_5d.get_fdata()    

data_4d_real = np.squeeze(data_5d)

new_img_4d = nb.Nifti1Image(data_4d_real, img_5d.affine)

nb.save(new_img_4d, output_4d_nii)


In [ ]:
#creating KxP hard assignment matrix for the anatomical thalamus segmentations

import Functional_Fusion.atlas_map as am
import HierarchBayesParcel.arrangements as ar
import nibabel as  nb
import numpy as np
import torch as pt

wk_dir = '/Users/incehusain/fs_projects'
iglesias_atlas_nii = f"{wk_dir}/dseg_segmentations_Language7T/sub_19_thalamus_dseg.nii.gz"
lut_file = f"{wk_dir}/compressionLookupTable.txt"
iglesias_head = '# Labels for thalamus parcellation using histological atlas (Iglesias et al.)'
lut_label_to_name = {}

with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name

thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)

img = nb.load(iglesias_atlas_nii)

data = img.get_fdata().astype(np.int32)

thalamus_voxel = data > 0

voxel_labels = data[thalamus_voxel]

P=len(voxel_labels)
#we are no longer reading it into MNISymThalamus1 space - going to make new ROI mask using anatomical images
#atlas, _ = am.get_atlas('MNISymThalamus1')
#U = atlas.read_data(iglesias_atlas_nii)
#P=U.shape[0]

label_to_index = {label: idx for idx, label in enumerate(thalamus_labels)}

prob_map_numpy = np.zeros((K, P))

for p, label in enumerate(voxel_labels):
    if label in label_to_index:
        k = label_to_index[label]
        prob_map_numpy[k, p] = 1.0

U_iglesias = np.array(prob_map_numpy)

U_iglesias_map = np.round(U_iglesias)
#ar_model = ar.build_arrangement_model(U_iglesias_map, prior_type='prob', atlas=atlas)
#print("TEST")


In [22]:
#Registering segmentations to MNI125NLin2009cSym 

import ants 
import os

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-19'

output_prefix = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_"

template = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"
T1_nii = f"{wk_dir}/freesurfer_Language7T/{sub}/mri/T1.mgz"   #or norm.mgz

seg_nii = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"

output_path = f"{wk_dir}"

template_ants = ants.image_read(template)
T1 = ants.image_read(T1_nii)
seg = ants.image_read(seg_nii)

print('Registering...')

registration = ants.registration(fixed=template_ants, moving=T1, type_of_transform='SyN', outprefix=output_prefix)

print('Registration done!')

warped_seg = ants.apply_transforms(fixed=template_ants, moving=seg, transformlist=registration['fwdtransforms'], interpolator='linear')

ants.image_write(warped_seg, f"{wk_dir}/pseg_segmentations_Language7T/{sub}_seg_warped.nii.gz")



Registering...
Registration done!


In [46]:
wk_dir = '/Users/incehusain/fs_projects'

thalamus_map_sub1 = np.load(f"{wk_dir}/pseg_matrices_Language7T/sub-01_pseg_matrix.npy")
thalamus_map_sub2 = np.load(f"{wk_dir}/pseg_matrices_Language7T/sub-02_pseg_matrix.npy")
thalamus_map_sub3 = np.load(f"{wk_dir}/pseg_matrices_Language7T/sub-04_pseg_matrix.npy")

print(thalamus_map_sub1.shape)

print(thalamus_map_sub2.shape)

print(thalamus_map_sub3[0].shape)


(58, 25064)
(58, 22172)
(25995,)
